# Orchestration Pattern — Notes and Runbook

This file captures notes for the Orchestration deployment in this repo.

## Purpose
- Orchestration pattern uses a central `saga-orchestrator` which issues commands to services and records saga instances.
- Services: `saga-orchestrator`, `order-service`, `inventory-service`, `payment-service`, `shipping-service`.

## Compose / Ports
From `saga_pattern/orchestration_pattern/compose.yaml`:
- MySQL: 3306
- Redis: 6379
- RabbitMQ: 5672 (AMQP), 15672 (management UI)
- saga-orchestrator: host port `8005` -> container `8005`
- order-service: host port `8011` -> container `8001` (note port mapping differs from choreography)
- inventory-service: container mapping likely `8002`, payment `8003`, shipping `8004` (confirm compose file)

## Start / Stop
Start the orchestration stack (build and detach):

```bash
cd saga_pattern/orchestration_pattern
docker-compose up -d --build
```

Stop and remove containers:

```bash
docker-compose down
```

## Quick smoke tests (host machine)
- Check Orchestrator health:

```bash
curl -v http://localhost:8005/health || curl -v http://localhost:8005/
```

- Create order via orchestrator (example):

```bash
curl -s -X POST http://localhost:8005/orders \
  -H 'Content-Type: application/json' \
  -d '{"customerId":"customer-001","bookId":"book-123","quantity":1}'
```

- Note: In `orchestration_pattern/compose.yaml`, the `order-service` is mapped to host port `8011:8001`. If earlier tests used `8001` for order-service, update tests to point to `8011` or change compose to use `8001`.

## Debugging tips
- If you get `404` to `http://localhost:8005/orders`, inspect the orchestrator logs:

```bash
docker-compose logs saga-orchestrator --tail=200
```

- Check service wiring: orchestrator environment variables `ORDER_SERVICE_URL`, etc., in the compose file should point to the internal docker hostnames and ports (container ports).
- Ensure RabbitMQ is healthy and the orchestrator is connected to it.

## Useful commands
- View RabbitMQ management UI: http://localhost:15672 (user/password from compose env)
- Run a shell in the orchestrator container for troubleshooting:

```bash
docker-compose exec saga-orchestrator /bin/sh
```


In [ ]:
# Test runner cell: run choreography and orchestration test scenarios and save CSV results
import requests
import time
import pandas as pd
import socket
from urllib.parse import urlparse
from pathlib import Path

out_csv = Path.cwd().parent / 'test_results.csv'  # saga_pattern/test_results.csv

# Service endpoints (matching compose mappings)
CHOREO_ORDER_URL = 'http://localhost:8001/orders'      # choreography order-service
ORCH_ORCHESTRATOR_URL = 'http://localhost:8005/orders' # orchestrator entry
ORCH_ORDER_HOST_PORT = 'http://localhost:8011/orders'  # orchestration's order-service mapped to 8011

def check_docker_running():
    # skip Docker check to avoid false negatives
    return True

def is_port_open(host: str, port: int, timeout: float = 1.0) -> bool:
    """Check if a TCP port is open on the given host."""
    try:
        with socket.create_connection((host, port), timeout=timeout):
            return True
    except Exception:
        return False

def wait_for_port(url: str, timeout: int = 60, interval: float = 1.0) -> bool:
    """Wait until the host:port in the URL accepts TCP connections."""
    parsed = urlparse(url)
    host = parsed.hostname or 'localhost'
    port = parsed.port or (443 if parsed.scheme == 'https' else 80)
    start = time.time()
    while time.time() - start < timeout:
        if is_port_open(host, port, timeout=1.0):
            return True
        time.sleep(interval)
    return False

# Define scenarios
scenarios = [
    ("choreography", "success", CHOREO_ORDER_URL, {"customerId":"customer-001","bookId":"book-123","quantity":1}),
    ("choreography", "stock_failure", CHOREO_ORDER_URL, {"customerId":"customer-002","bookId":"book-456","quantity":1}),
    ("choreography", "payment_failure", CHOREO_ORDER_URL, {"customerId":"customer-004","bookId":"book-789","quantity":1}),
    ("orchestration", "orchestrator_submit", ORCH_ORCHESTRATOR_URL, {"customerId":"customer-001","bookId":"book-123","quantity":1}),
    ("orchestration", "order_service_direct", ORCH_ORDER_HOST_PORT, {"customerId":"customer-001","bookId":"book-123","quantity":1}),
]

results = []

# Run scenarios with port wait and unreachable handling
for pattern, name, url, payload in scenarios:
    parsed = urlparse(url)
    host = parsed.hostname or 'localhost'
    port = parsed.port or (443 if parsed.scheme == 'https' else 80)

    # Measure wait duration separately
    wait_start = time.time()
    reachable = wait_for_port(url, timeout=60, interval=1.0)

    if not reachable:
        elapsed = time.time() - wait_start
        results.append({
            'pattern': pattern,
            'scenario': name,
            'url': url,
            'status_code': 'UNREACHABLE',
            'response_time': elapsed,
            'result': f'Host {host}:{port} not reachable within timeout'
        })
        continue

    # If reachable, measure HTTP request latency only
    req_start = time.time()
    try:
        r = requests.post(url, json=payload, timeout=10)
        elapsed = time.time() - req_start
        try:
            body = r.json()
        except Exception:
            body = r.text
        status_code = r.status_code
    except requests.exceptions.RequestException as e:
        elapsed = time.time() - req_start
        status_code = getattr(getattr(e, 'response', None), 'status_code', 'N/A')
        body = str(e)
    results.append({
        'pattern': pattern,
        'scenario': name,
        'url': url,
        'status_code': status_code,
        'response_time': elapsed,
        'result': body
    })

# Build DataFrame and save CSV
expected_cols = ['pattern', 'scenario', 'url', 'status_code', 'response_time', 'result']
df = pd.DataFrame(results, columns=expected_cols)
df['result'] = df['result'].apply(lambda x: str(x) if x is not None else '')
out_csv.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(out_csv, index=False)
print('Wrote test results to', out_csv)
print(df)

In [ ]:
# DB initialization helper: bring up the chosen stack and initialize MySQL
import subprocess
import time
from pathlib import Path

# Choose pattern: 'choreography' or 'orchestration'
pattern = 'orchestration'  # change if you want choreography
compose_dir = Path.cwd() / f"{pattern}_pattern"
print('Compose dir:', compose_dir)

def run(cmd, cwd=None, timeout=60):
    print('RUN:', cmd)
    p = subprocess.run(cmd, cwd=cwd, shell=True, capture_output=True, text=True, timeout=timeout)
    print('returncode', p.returncode)
    if p.stdout:
        print('STDOUT:', p.stdout[:1000])
    if p.stderr:
        print('STDERR:', p.stderr[:1000])
    return p

# Start compose
run('docker-compose up -d --build', cwd=str(compose_dir))

# Wait for MySQL to be healthy (poll)
healthy = False
for i in range(30):
    try:
        p = run("docker-compose exec -T mysql mysqladmin ping -h localhost -ucloudmart_user -pcloudmart_pass", cwd=str(compose_dir))
        if p.returncode == 0 and 'mysqld is alive' in (p.stdout + p.stderr):
            healthy = True
            print('MySQL is healthy')
            break
    except Exception as e:
        print('poll exception', e)
    time.sleep(2)

if not healthy:
    print('Warning: MySQL did not report healthy within timeout. Proceeding to attempt initialization anyway.')

# Run init script inside container (if present)
init_path_in_container = '/docker-entrypoint-initdb.d/init.sql'
cmd_init = f'docker-compose exec -T mysql sh -c "mysql -ucloudmart_user -pcloudmart_pass cloudmart_saga < {init_path_in_container}"'
init_res = run(cmd_init, cwd=str(compose_dir))
print('Init exit code:', init_res.returncode)

print('DB initialization step finished for', pattern)
